In [13]:
import os
import gc

import pandas as pd
import numpy as np
import torch

from transformers import PatchTSTForPrediction

from torch.utils.data import DataLoader
from datasets import Dataset

In [14]:
data = "coin"

output_dir = "saved_models"
log_dir = os.path.join('logstf', data)

loss_name = "mse"

num_train_epochs = 300
model_num = 1
model_path = "./saved_models"
learning_rate = 1e-7

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
## target domain
target_X = pd.read_csv(f"../data/{data}/train_input_7.csv").iloc[:, 1:].values.astype(np.float32)
target_y = pd.read_csv(f"../data/{data}/train_output_7.csv").iloc[:, 1:].values.astype(np.float32)

target_X_val = target_X[-round(target_X.shape[0] * 0.2):, :].astype(np.float32)
target_y_val = target_y[-round(target_y.shape[0] * 0.2):].astype(np.float32)
target_X = target_X[:-round(target_X.shape[0] * 0.2), :].astype(np.float32)
target_y = target_y[:-round(target_y.shape[0] * 0.2)].astype(np.float32)

test_X  = pd.read_csv(f"../data/{data}/val_input_7.csv").iloc[:, 1:].values.astype(np.float32)
test_y  = pd.read_csv(f"../data/{data}/val_output_7.csv").iloc[:, 1:].values.astype(np.float32)

In [16]:
def array_to_dataset(X, y):
    X, y = torch.tensor(X), torch.tensor(y)
    X = X.reshape(-1, X.shape[1], 1)
    y = y.reshape(-1, y.shape[1], 1)

    dataset = torch.utils.data.TensorDataset(X, y)

    return dataset

train_dataset = array_to_dataset(target_X, target_y)
val_dataset = array_to_dataset(target_X_val, target_y_val)
test_dataset = array_to_dataset(test_X, test_y)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 64)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 64)

In [ ]:
for k in range(1, model_num+1):
    current_path = os.path.join(model_path, f"model_{loss_name}_{k}.pth")

    backbone_model = PatchTSTForPrediction.from_pretrained(os.path.join(model_path, "PatchTSTBackbone")).to(device)
    backbone_model.load_state_dict(torch.load(current_path))    ## 구조 변경 없이 그대로 로드

In [ ]:
optimizer = torch.optim.AdamW(backbone_model.parameters(), lr = learning_rate)
log_data = []

if loss_name == "mse":
    loss_fn = torch.nn.MSELoss()
elif loss_name == "mae":
    loss_fn = torch.nn.L1Loss()
# elif loss_name

## early stopping
PATIENCE = 10
best_val_loss = np.inf
patience_counter = 0

for epoc in range(num_train_epochs):
    backbone_model.train()

    total_train_loss = 0

    ys = []
    yhats = []

    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        yhat = backbone_model(X).prediction_outputs
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()

        ys.append(y)
        yhats.append(yhat)

    avg_train_loss = loss_fn(yhat, y).item()

    backbone_model.eval()

    with torch.no_grad():
        yys = []
        yyhats = []

        for XX, yy in val_dataloader:
            XX = XX.to(device)
            yys.append(yy.to(device))
            yyhats.append(backbone_model(XX).prediction_outputs)

        yyhat = torch.concat(yyhats)
        yy = torch.concat(yys)

        val_loss = loss_fn(yyhat, yy)

    print(f"Epoch {epoc+1}/{num_train_epochs} | Train Loss: {avg_train_loss:.6f}\t\t Val Loss: {val_loss:.6f}")

    log_data.append({"epoch": epoc, "loss": avg_train_loss, "eval_loss": val_loss.item()})

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state_dict = backbone_model.state_dict()   ## 저장 없이 결과물만 산출...
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        break

Epoch 1/300 | Train Loss: 30.743526		 Val Loss: 7.494946
Epoch 2/300 | Train Loss: 287.917725		 Val Loss: 7.441466
Epoch 3/300 | Train Loss: 37.383850		 Val Loss: 7.738950
Epoch 4/300 | Train Loss: 35.921658		 Val Loss: 7.342999
Epoch 5/300 | Train Loss: 23.675152		 Val Loss: 7.153208
Epoch 6/300 | Train Loss: 2452.205078		 Val Loss: 7.131973
Epoch 7/300 | Train Loss: 77.789261		 Val Loss: 7.370630
Epoch 8/300 | Train Loss: 49.830811		 Val Loss: 7.192869
Epoch 9/300 | Train Loss: 60.322872		 Val Loss: 7.164845
Epoch 10/300 | Train Loss: 49.355888		 Val Loss: 7.070136
Epoch 11/300 | Train Loss: 174.342743		 Val Loss: 6.989264
Epoch 12/300 | Train Loss: 623.590088		 Val Loss: 7.280973
Epoch 13/300 | Train Loss: 2447.274414		 Val Loss: 7.187242
Epoch 14/300 | Train Loss: 170.726242		 Val Loss: 6.730597
Epoch 15/300 | Train Loss: 70.426926		 Val Loss: 7.237209
Epoch 16/300 | Train Loss: 5.534828		 Val Loss: 7.214797
Epoch 17/300 | Train Loss: 36.980663		 Val Loss: 6.975815
Epoch 18/300 | T

In [19]:
backbone_model.load_state_dict(best_state_dict)

<All keys matched successfully>

In [26]:
pd.DataFrame(log_data).to_csv(os.path.join(log_dir, f"transfer_{loss_name}_lr{learning_rate}_run{1}.csv"))

In [20]:
with torch.no_grad():
    yys = []
    yyhats = []

    for XX, yy in test_dataloader:
        XX = XX.to(device)
        yys.append(yy.to(device))
        yyhats.append(backbone_model(XX).prediction_outputs)

    yyhat = torch.concat(yyhats)
    yy = torch.concat(yys)

    test_loss = loss_fn(yyhat, yy)

In [21]:
mseLoss = torch.nn.MSELoss()
maeLoss = torch.nn.L1Loss()

def smape(yy, yyhat):
    numerator = 100*abs(yy - yyhat)
    denominator = (abs(yy) + abs(yyhat))/2
    smape = torch.mean(numerator / denominator)
    return smape

print(f"test RMSE: {torch.sqrt(mseLoss(yyhat, yy))}")
print(f"test MAE: {maeLoss(yyhat, yy)}")
print(f"test SMAPE: {smape(yy, yyhat)}")

test RMSE: 3.8376009464263916
test MAE: 1.6058361530303955
test SMAPE: 2.2649388313293457


In [8]:
with torch.no_grad():
    yys = []
    yyhats = []

    for XX, yy in test_dataloader:
        XX = XX.to(device)
        yys.append(yy.to(device))
        yyhats.append(backbone_model(XX).prediction_outputs)

    yyhat = torch.concat(yyhats)
    yy = torch.concat(yys)

    test_loss = loss_fn(yyhat, yy)

In [9]:
mseLoss = torch.nn.MSELoss()
maeLoss = torch.nn.L1Loss()

def smape(yy, yyhat):
    numerator = 100*abs(yy - yyhat)
    denominator = (abs(yy) + abs(yyhat))/2
    smape = torch.mean(numerator / denominator)
    return smape

print(f"test RMSE: {torch.sqrt(mseLoss(yyhat, yy))}")
print(f"test MAE: {maeLoss(yyhat, yy)}")
print(f"test SMAPE: {smape(yy, yyhat)}")

test RMSE: 3.8342514038085938
test MAE: 1.5989410877227783
test SMAPE: 2.2572214603424072


> ????????? 그냥 단순한 Linear head인데...

In [29]:
backbone_model.head

PatchTSTPredictionHead(
  (flatten): Flatten(start_dim=2, end_dim=-1)
  (projection): Linear(in_features=1792, out_features=24, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)